In [1]:
from __future__ import absolute_import, division, print_function

import pickle
import sys
import numpy as np
import torch

import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

import os
print("before: %s"%os.getcwd())
os.chdir("/Users/clarasong/Research/exercise_model")
print("after: %s"%os.getcwd())

# import vector3d
%matplotlib notebook

before: /Users/clarasong/Research/exercise_model
after: /Users/clarasong/Research/exercise_model


In [2]:
def distance(x1, x2):
    a = np.sqrt(np.sum(np.square(x1 - x2)))
    return np.sqrt(np.sum(np.square(x1 - x2)))

def rotate(kps):
    t = (kps[11] + kps[14]) / 2  # translation vector
    d = distance(kps[11], kps[14]) / 2  # half of distance between left shoulder and right shoulder
    r1 = (kps[11] - t) / d
    b = distance((kps[11] + kps[14]) / 2, kps[7])  # distance between the center of she shoulder axis and the spine
    r3 = 1 / -b * (kps[7] - t)
    r2 = np.cross(r1, r3)
    R = np.vstack((r1, r2, r3)).T  # rotation matrix
    R_inv = np.linalg.inv(R)  # R^(-1)

    left = pd.Series([11, 12, 13, 4, 5, 6, 7, 8, 9, 10, 0])
    right = pd.Series([14, 15, 16, 1, 2, 3])
    
    for i in range(kps.shape[0]):
        kps[i] = np.matmul(R_inv, (kps[i] - t))
        if (left.isin([i]).any()):
            kps[i] = kps[i] - kps[11]
        else: 
            kps[i] = kps[i] - kps[14]
            
    return kps

In [3]:
def processing_video_data_raw(data, exc_name_str):
    kps_3d = data['kps_3d']

    v_w_x_left = [] # video_wrist_x_left_arm
    v_w_y_left = []
    v_w_z_left = []
    v_l_x_left = []
    v_l_y_left = []
    v_l_z_left = []
    v_s_x_left = []
    v_s_y_left = []
    v_s_z_left = []

    v_w_x_right = []# video_wrist_x_right_arm
    v_w_y_right = []
    v_w_z_right = []
    v_l_x_right = []
    v_l_y_right = []
    v_l_z_right = []
    v_s_x_right = []
    v_s_y_right = []
    v_s_z_right = []
    
    v_c_x_left = []
    v_c_y_left = []
    v_c_z_left = []

    v_c_x_right = []
    v_c_y_right = []
    v_c_z_right = []

    for i, kps in enumerate(kps_3d):
        #kps = rotate(kps)
        #left
        v_s_x_left.append(kps[11][0])
        v_s_y_left.append(kps[11][1])
        v_s_z_left.append(kps[11][2])
        v_l_x_left.append(kps[12][0])
        v_l_y_left.append(kps[12][1])
        v_l_z_left.append(kps[12][2])
        v_w_x_left.append(kps[13][0])
        v_w_y_left.append(kps[13][1])
        v_w_z_left.append(kps[13][2])

        #right
        v_s_x_right.append(kps[14][0])
        v_s_y_right.append(kps[14][1])
        v_s_z_right.append(kps[14][2])
        v_l_x_right.append(kps[15][0])
        v_l_y_right.append(kps[15][1])
        v_l_z_right.append(kps[15][2])
        v_w_x_right.append(kps[16][0])
        v_w_y_right.append(kps[16][1])
        v_w_z_right.append(kps[16][2])
        #center 
        v_c_x_right.append(kps[7][0])
        v_c_y_right.append(kps[7][1])
        v_c_z_right.append(kps[7][2])
        
    #data_all = [s_x_left, s_y_left, s_z_left, l_x_left, l_y_left, l_z_left, w_x_left, w_y_left, w_z_left]
    ##data_all = [v_l_x_right, v_l_y_right, v_l_z_right, v_w_x_right, v_w_y_right, v_w_z_right]
    #data_all = [train_elb_x, train_elb_y, train_elb_z, train_wst_x, train_wst_y, train_wst_z]
    
    #v_l_y_left = [-i for i in v_l_y_left] # rotation y axis
    #v_w_y_left = [-i for i in v_w_y_left]
    #v_s_y_left = [-i for i in v_s_y_left]
   #
    #v_l_y_right = [-i for i in v_l_y_right] # rotation y axis
    #v_w_y_right = [-i for i in v_w_y_right]
    #v_s_y_right = [-i for i in v_s_y_right]
  #
    #v_c_y_right = [-i for i in v_c_y_right]
    
    data_all = [
                v_l_x_right, v_l_y_right, v_l_z_right, 
                v_w_x_right, v_w_y_right, v_w_z_right, 
                v_s_x_right, v_s_y_right, v_s_z_right,
                
                v_c_x_right, v_c_y_right, v_c_z_right, 
                
                v_s_x_left, v_s_y_left, v_s_z_left,
                v_l_x_left, v_l_y_left, v_l_z_left, 
                v_w_x_left, v_w_y_left, v_w_z_left         
               ]
    

    # left을 사용해야함 카메라 기준으로left는 사람의 오른손. ?? 이거 틀림? 틀림. 
    d = {'e_x_right':data_all[0], 'e_y_right': data_all[1], 'e_z_right':data_all[2], 
         'w_x_right':data_all[3], 'w_y_right': data_all[4] ,'w_z_right':data_all[5], 
         's_x_right':data_all[6], 's_y_right': data_all[7], 's_z_right':data_all[8],
         'c_x_right':data_all[9], 'c_y_right': data_all[10], 'c_z_right':data_all[11],
         's_x_left':data_all[12], 's_y_left': data_all[13], 's_z_left':data_all[14],
         'e_x_left':data_all[15], 'e_y_left': data_all[16], 'e_z_left':data_all[17], 
         'w_x_left':data_all[18], 'w_y_left': data_all[19] ,'w_z_left':data_all[20], 
         
        }
    v_exc = pd.DataFrame(data = d)
    v_exc['types'] = exc_name_str
    return v_exc

In [4]:
class Keypoints3D(object): # 카메라 기준으로 왼쪽 오른쪽???
    HIP = 0
    R_HIP = 1
    R_KNEE = 2
    R_FOOT = 3
    L_HIP = 4
    L_KNEE = 5
    L_FOOT = 6
    
    SPINE = 7
    THORAX = 8
    NOSE = 9
    HEAD = 10
    
    L_SHOULDER = 11
    L_ELBOW = 12
    L_WRIST = 13
    R_SHOULDER = 14
    R_ELBOW = 15
    R_WRIST = 16

def processing_video_data(data, exc_name_str):
    kps_3d = data['kps_3d']

    v_w_x_left = [] # video_wrist_x_left_arm
    v_w_y_left = []
    v_w_z_left = []
    v_l_x_left = []
    v_l_y_left = []
    v_l_z_left = []
    v_s_x_left = []
    v_s_y_left = []
    v_s_z_left = []

    v_w_x_right = []# video_wrist_x_right_arm
    v_w_y_right = []
    v_w_z_right = []
    v_l_x_right = []
    v_l_y_right = []
    v_l_z_right = []
    v_s_x_right = []
    v_s_y_right = []
    v_s_z_right = []
    
    v_c_x_left = []
    v_c_y_left = []
    v_c_z_left = []

    v_c_x_right = []
    v_c_y_right = []
    v_c_z_right = []

    for i, kps in enumerate(kps_3d):
        kps = rotate(kps)
        #left
        v_s_x_left.append(kps[11][0])
        v_s_y_left.append(kps[11][1])
        v_s_z_left.append(kps[11][2])
        v_l_x_left.append(kps[12][0])
        v_l_y_left.append(kps[12][1])
        v_l_z_left.append(kps[12][2])
        v_w_x_left.append(kps[13][0])
        v_w_y_left.append(kps[13][1])
        v_w_z_left.append(kps[13][2])

        #right
        v_s_x_right.append(kps[14][0])
        v_s_y_right.append(kps[14][1])
        v_s_z_right.append(kps[14][2])
        v_l_x_right.append(kps[15][0])
        v_l_y_right.append(kps[15][1])
        v_l_z_right.append(kps[15][2])
        v_w_x_right.append(kps[16][0])
        v_w_y_right.append(kps[16][1])
        v_w_z_right.append(kps[16][2])
        #center 
        v_c_x_right.append(kps[7][0])
        v_c_y_right.append(kps[7][1])
        v_c_z_right.append(kps[7][2])
        
    #data_all = [s_x_left, s_y_left, s_z_left, l_x_left, l_y_left, l_z_left, w_x_left, w_y_left, w_z_left]
    ##data_all = [v_l_x_right, v_l_y_right, v_l_z_right, v_w_x_right, v_w_y_right, v_w_z_right]
    #data_all = [train_elb_x, train_elb_y, train_elb_z, train_wst_x, train_wst_y, train_wst_z]
    
    #v_l_y_left = [-i for i in v_l_y_left] # rotation y axis
    #v_w_y_left = [-i for i in v_w_y_left]
    #v_s_y_left = [-i for i in v_s_y_left]
   #
    #v_l_y_right = [-i for i in v_l_y_right] # rotation y axis
    #v_w_y_right = [-i for i in v_w_y_right]
    #v_s_y_right = [-i for i in v_s_y_right]
  #
    #v_c_y_right = [-i for i in v_c_y_right]
    
    data_all = [
                v_l_x_right, v_l_y_right, v_l_z_right, 
                v_w_x_right, v_w_y_right, v_w_z_right, 
                v_s_x_right, v_s_y_right, v_s_z_right,
                
                v_c_x_right, v_c_y_right, v_c_z_right, 
                
                v_s_x_left, v_s_y_left, v_s_z_left,
                v_l_x_left, v_l_y_left, v_l_z_left, 
                v_w_x_left, v_w_y_left, v_w_z_left         
               ]
    

    # left을 사용해야함 카메라 기준으로left는 사람의 오른손. ?? 이거 틀림? 틀림. 
    d = {'e_x_right':data_all[0], 'e_y_right': data_all[1], 'e_z_right':data_all[2], 
         'w_x_right':data_all[3], 'w_y_right': data_all[4] ,'w_z_right':data_all[5], 
         's_x_right':data_all[6], 's_y_right': data_all[7], 's_z_right':data_all[8],
         'c_x_right':data_all[9], 'c_y_right': data_all[10], 'c_z_right':data_all[11],
         's_x_left':data_all[12], 's_y_left': data_all[13], 's_z_left':data_all[14],
         'e_x_left':data_all[15], 'e_y_left': data_all[16], 'e_z_left':data_all[17], 
         'w_x_left':data_all[18], 'w_y_left': data_all[19] ,'w_z_left':data_all[20], 
         
        }
    v_exc = pd.DataFrame(data = d)
    v_exc['types'] = exc_name_str
    return v_exc

In [ ]:
def group_frames(df, frames_per_sample):
    """
    Args:
        df (Dataframe)
        frames_per_sample (int): number of frames group together for one sameple
    """
    length = df.keys[0].size
    
    for sample in range(0, length/frames_per_sample, frames_per_sample):
        

In [91]:
pose_file = './1_pose_3d.cpl'
test = pickle.load(open(pose_file, 'rb'))
df_01 = processing_video_data(test, 'exc01')

df_test_01 = df_01.iloc[0:100, :]     # frames 0..100 is test data
df_train_01 = df_01.iloc[100:900, :]  # frames 100..900 is training data (guess that after 900 is when angled videos begin)

print(df_01["e_x_right"].size)
# print(df)

1277


In [88]:
# pose_file = './2_pose_3d.cpl'
# test = pickle.load(open(pose_file, 'rb'))
# processing_video_data(test, 'exc02')

In [90]:
pose_file = './4_pose_3d.cpl'
test = pickle.load(open(pose_file, 'rb'))
df_04 = processing_video_data(test, 'exc04')

df_test_04 = df_04.iloc[0:100, :]
df_train_04 = df_04.iloc[100:900, :]

print(df_04["e_x_right"].size)

1244


In [94]:
df_test_frames = [df_test_01, df_test_04]
df_train_frames = [df_train_01, df_train_04]

df_test = pd.concat(df_test_frames)
df_train = pd.concat(df_train_frames)

df_test.to_csv(r'./data/test1.csv')
df_train.to_csv(r'./data/train1.csv')